In [1]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)

In [2]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""
import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy import ndimage
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.stats import median_absolute_deviation
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib


import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat

calc_emergence = True

debug=False
extra_layers=True

csv_ending = '_mb_bins.csv'
outdir_csv = debris_prms.mb_binned_fp
if os.path.exists(outdir_csv) == False:
    os.makedirs(outdir_csv)

In [3]:
# Debris cover extent shapefile with statistics
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])

# Subset by percent debris-covered or debris-covered area
dc_shp_subset = dc_shp[((dc_shp['DC_Area__1'] > debris_prms.dc_percarea_threshold) | 
                        (dc_shp['DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold))
                        & (dc_shp['Area'] > debris_prms.min_glac_area)].copy()
dc_shp_subset.reset_index(inplace=True, drop=True)
dc_shp_subset['CenLon_360'] = dc_shp_subset['CenLon']
dc_shp_subset.loc[dc_shp_subset['CenLon_360'] < 0, 'CenLon_360'] = (
    360 + dc_shp_subset.loc[dc_shp_subset['CenLon_360'] < 0, 'CenLon_360'])
# dc_shp_subset

rgiid_list = [x.split('-')[1] for x in dc_shp_subset['RGIId'].values]
main_glac_rgi_subset = debris_prms.selectglaciersrgitable(rgiid_list)
main_glac_rgi_subset

598 glaciers in region 17 are included in this model run: ['00023', '00080', '00141', '00172', '00466', '00516', '01019', '01100', '01101', '01108', '01112', '01123', '01128', '01218', '01236', '01253', '01421', '01429', '01448', '01495', '01506', '01554', '01638', '01687', '01762', '01879', '01917', '01921', '01940', '01970', '01972', '01974', '02025', '02035', '02071', '02265', '02427', '02504', '02519', '02531', '02547', '02600', '02617', '02642', '02665', '02666', '02671', '02674', '02685', '02687'] and more
This study is focusing on 598 glaciers in region [17]


,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Form,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float
GlacNo,,,,,,,,,,,,,,,,,,,,
0,22,RGI60-17.00023,-70.1612,-34.5673,17,2,2.893,3346,3831,3618,11.1,63,2774,0,0,9,20019999,23,17.00023,17.00023
1,79,RGI60-17.00080,-70.3269,-34.9967,17,2,8.805,2989,3855,3479,13.9,132,3886,0,0,9,20019999,80,17.00080,17.00080
2,140,RGI60-17.00141,-73.0097,-50.9444,17,1,2.254,909,2173,1147,22.1,20,3972,0,0,9,20019999,141,17.00141,17.00141
3,171,RGI60-17.00172,-73.4697,-49.8866,17,1,838.044,193,3029,1347,7.5,133,59615,0,2,9,20019999,172,17.00172,17.00172
4,465,RGI60-17.00466,-70.5760,-35.2750,17,2,13.681,2937,4061,3499,15.7,147,3631,0,0,9,20019999,466,17.00466,17.00466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,15897,RGI60-17.15898,-73.5690,-47.1840,17,1,268.749,28,3277,1256,7.0,248,46729,0,2,9,20011103,15898,17.15898,17.15898
594,15898,RGI60-17.15899,-73.7160,-47.1390,17,1,153.237,50,2063,985,10.4,267,24458,0,0,9,20011103,15899,17.15899,17.15899
595,15902,RGI60-17.15903,-73.6700,-47.0460,17,1,161.389,16,2401,945,10.8,272,31214,0,2,9,20011103,15903,17.15903,17.15903


In [4]:
if os.path.exists(debris_prms.dhdt_vel_fns_fp) == False:
    os.makedirs(debris_prms.dhdt_vel_fns_fp)
dhdt_vel_fns_fn = debris_prms.dhdt_vel_fns_fn.replace('XXXX',debris_prms.roi)
if os.path.exists(debris_prms.dhdt_vel_fns_fp + dhdt_vel_fns_fn):
    dhdt_vel_fns_df = pd.read_csv(debris_prms.dhdt_vel_fns_fp + dhdt_vel_fns_fn)
else:
    dhdt_vel_fns_df = pd.DataFrame(np.zeros((main_glac_rgi_subset.shape[0], 3)),
                                   columns=['RGIId', 'dhdt_fullfn', 'vel_fullfn'])
    dhdt_vel_fns_df['RGIId'] = main_glac_rgi_subset['RGIId']

In [5]:
# np.where(main_glac_rgi_subset.rgino_str == '11.03005')

In [6]:
# ===== PROCESS EACH GLACIER =====
for nglac, glac_idx in enumerate(main_glac_rgi_subset.index.values):
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[0]]):
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[120]]): # Miage
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[2307]]): # Ngozumpa

    glac_str = main_glac_rgi_subset.loc[glac_idx,'rgino_str']
    rgiid = main_glac_rgi_subset.loc[glac_idx,'RGIId']
    region = glac_str.split('.')[0]

    if int(region) < 10:
        glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
    else:
        glac_str_noleadzero = glac_str

    if os.path.exists(debris_prms.hd_fp + debris_prms.hd_fn_sample.replace('XXXX',glac_str_noleadzero)) == False:

        print(nglac, glac_idx, rgiid)

        # Create glacier feature from ice thickness raster
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
        
        if os.path.exists(thick_dir + thick_fn):
        
            gf = create_glacfeat(thick_dir, thick_fn)

            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if os.path.exists(dc_shp_proj_fn) == False:
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()


            # ==== CHECK IF TIF HAS DHDT DATA OVER THE GLACIER =====
            mb_fullfns = []
            find_mb = True
            dhdt_fn_wglacier = None
            for mb_fp in debris_prms.mb_fp_list_roi[debris_prms.roi]:
                if find_mb:
                    for i in os.listdir(mb_fp):
                        if i.endswith('.tif'):
                            mb_fullfns.append(mb_fp + i)
                    tif_count = 0
                    while find_mb and tif_count < len(mb_fullfns):
                        dhdt_fn = mb_fullfns[tif_count]
                        if debug:
                            print(tif_count, dhdt_fn.split('/')[-1])

                        # Add the filenames
                        fn_dict = OrderedDict()
                        # DEM
                        z1_fp = debris_prms.oggm_fp + 'dems/RGI60-' + str(region.zfill(2)) + '/'
                        z1_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_dem.tif'
                        fn_dict['z1'] = z1_fp + z1_fn
                        # Ice thickness
                        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
                        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
                        fn_dict['ice_thick'] = thick_dir + thick_fn
                        # dh/dt
                        fn_dict['dhdt'] = dhdt_fn
                        # ===== PROCESS THE DATA =====
                        #Expand extent to include buffered region around glacier polygon
                        warp_extent = geolib.pad_extent(gf.glac_geom_extent, width=debris_prms.buff_dist)
                        #Warp everything to common res/extent/proj
                        z1_gt = gdal.Open(fn_dict['z1']).GetGeoTransform()
                        z1_res = np.min([z1_gt[1], -z1_gt[5]])
                        ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                                           t_srs=gf.aea_srs, verbose=False, r='cubic')
                        ds_dict = dict(zip(fn_dict.keys(), ds_list))
                        gf.ds_dict = ds_dict

                        if 'z1' in ds_dict:
                            #This is False over glacier polygon surface, True elsewhere - can be applied directly
                            glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1'])
                            glac_geom_mask_copy = glac_geom_mask.copy()
                            gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
                            # Debris cover
                            dc_shp_lyr_mask = geolib.lyr2mask(dc_shp_lyr, ds_dict['ice_thick'])
                            gf.dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
                            if 'dhdt' in ds_dict:
                                gf.dhdt = np.ma.array(iolib.ds_getma(ds_dict['dhdt']), mask=glac_geom_mask_copy)
                                gf.dhdt.mask = np.ma.mask_or(
                                    glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.dhdt.data, 
                                                                                     np.isnan(gf.dhdt.data))))
                                gf.dc_dhdt = np.ma.array(iolib.ds_getma(ds_dict['dhdt']), mask=glac_geom_mask_copy)
                                gf.dc_dhdt.mask = gf.dc_mask
                                gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=gf.dc_mask)
                                # Count dhdt pixels
                                dc_dhdt_pixels = len(gf.dc_dhdt.nonzero()[0])
                                if dc_dhdt_pixels / gf.dc_area.count() * 100 > 75:
                                    dhdt_fn_wglacier = dhdt_fn
                                    find_mb = False
                                    if debug:
                                        print('\n# z1 dc pixels:', gf.dc_area.count())
                                        print('# dc_dhdt_pixels:', dc_dhdt_pixels)
                                        var_full2plot = gf.dhdt.copy()
                                        clim = malib.calcperc(var_full2plot, (2,98))
                                        plot_array(var_full2plot, clim, [glac_str + ' dhdt'], 'inferno', 'dhdt (m/yr)', 
                                                   close_fig=False)
                        # Loop over layers        
                        tif_count += 1

            # ==== CHECK IF VELOCITY DATA OVER THE GLACIER =====
            vel_fullfns = []
            find_vel = True
            vx_fn_wglacier = None
            if find_vel and dhdt_fn_wglacier is not None:
                vx_fns = debris_prms.vx_dir_dict_list[debris_prms.roi]
                tif_count = 0
                while find_vel and tif_count < len(vx_fns):
                    vx_fn = vx_fns[tif_count]

                    if debug:
                        print(tif_count, vx_fn.split('/')[-1])

                    # Add the filenames
                    # Velocity
                    fn_dict['vx'] = vx_fn
                    fn_dict['vy'] = vx_fn.replace('_vx', '_vy')
                    # ===== PROCESS THE DATA =====
                    ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                                       t_srs=gf.aea_srs, verbose=False, r='cubic')
                    ds_dict = dict(zip(fn_dict.keys(), ds_list))
                    gf.ds_dict = ds_dict
                    if 'vx' in ds_dict and 'vy' in ds_dict:
                        #Load surface velocity maps
                        gf.vx = np.ma.array(iolib.ds_getma(ds_dict['vx']), mask=glac_geom_mask)
                        gf.vy = np.ma.array(iolib.ds_getma(ds_dict['vy']), mask=glac_geom_mask)
                        gf.vm = np.ma.sqrt(gf.vx**2 + gf.vy**2)
                        gf.dc_vm = gf.vm.copy()
                        gf.dc_vm.mask = gf.dc_mask
                        # Count velocity pixels
                        dc_vel_pixels = len(gf.dc_vm.nonzero()[0])
                        if debug:
                                print('\n# z1 dc pixels:', gf.dc_area.count())
                                print('# dc vel_pixels:', dc_vel_pixels)
                                var_full2plot = gf.vm.copy()
                                clim = malib.calcperc(var_full2plot, (2,98))
                                plot_array(var_full2plot, clim, [glac_str + ' velocity'], 'inferno', 'vm (m/yr)', 
                                           close_fig=False)
                        if dc_vel_pixels / gf.dc_area.count() * 100 > 75:
                            vx_fn_wglacier = vx_fn
                            find_vel = False

                    # Loop over layers        
                    tif_count += 1


            # ===== Add layers =====
            if dhdt_fn_wglacier is not None and vx_fn_wglacier is not None:
                gf.add_layers(dc_shp_lyr, gf_add_dhdt=True, dhdt_fn=dhdt_fn_wglacier, gf_add_vel=True, vx_fn=vx_fn_wglacier, 
                              gf_add_ts=False, gf_add_slope_aspect=True, gf_add_ts_info=False, calc_emergence=True, 
                              debug_emergence=False)
                # Save dhdt and vel filenames
                dhdt_vel_fns_df.loc[glac_idx,:] = [rgiid, dhdt_fn_wglacier, vx_fn_wglacier]

                # ===== PLOTS =====
    #             plot_layers = True
                plot_layers = False
                if plot_layers:
                    # DEM
                    var_full2plot = gf.z1.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
                    # Elevation change
                    var_full2plot = gf.dhdt.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' dhdt'], 'inferno', 'dhdt (m/yr)', close_fig=False)
                    # Velocity
                    var_full2plot = gf.vm.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' velocity'], 'inferno', 'vel (m/yr)', close_fig=False)
                    # Emergence velocity
                    if gf.emvel is not None:
                        var_full2plot = gf.emvel.copy()
                        clim = malib.calcperc(var_full2plot, (2,98))
                        plot_array(var_full2plot, clim, [glac_str + ' emvel'], 'inferno', 'emvel (m/yr)', close_fig=False)
                    # Surface temperature
                    if gf.ts is not None:
                        var_full2plot = gf.ts.copy()
                        clim = malib.calcperc(var_full2plot, (2,98))
                        plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)

                # Bin data
                outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                # Export binned data
                if int(gf.feat_fn.split('.')[0]) < 10:
                    outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:7] + csv_ending)
                else:
                    outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:8] + csv_ending)
                outbins_df.loc[:,:] = np.nan_to_num(outbins_df.loc[:,:],0)
                outbins_df.to_csv(outbins_fullfn, index=False)

# Save updated filenames
dhdt_vel_fns_df.to_csv(debris_prms.dhdt_vel_fns_fp + dhdt_vel_fns_fn, index=False)

0 0 RGI60-17.00023
1 1 RGI60-17.00080
2 2 RGI60-17.00141
3 3 RGI60-17.00172
4 4 RGI60-17.00466
5 5 RGI60-17.00516
6 6 RGI60-17.01019
7 7 RGI60-17.01100
8 8 RGI60-17.01101
9 9 RGI60-17.01108
10 10 RGI60-17.01112
11 11 RGI60-17.01123
12 12 RGI60-17.01128
13 13 RGI60-17.01218
14 14 RGI60-17.01236
15 15 RGI60-17.01253
16 16 RGI60-17.01421
17 17 RGI60-17.01429
18 18 RGI60-17.01448
19 19 RGI60-17.01495
20 20 RGI60-17.01506
21 21 RGI60-17.01554
22 22 RGI60-17.01638
23 23 RGI60-17.01687
24 24 RGI60-17.01762
25 25 RGI60-17.01879
26 26 RGI60-17.01917
27 27 RGI60-17.01921
28 28 RGI60-17.01940
29 29 RGI60-17.01970
30 30 RGI60-17.01972
31 31 RGI60-17.01974
32 32 RGI60-17.02025
33 33 RGI60-17.02035
34 34 RGI60-17.02071
35 35 RGI60-17.02265
36 36 RGI60-17.02427
37 37 RGI60-17.02504
38 38 RGI60-17.02519
39 39 RGI60-17.02531
40 40 RGI60-17.02547
41 41 RGI60-17.02600
42 42 RGI60-17.02617
43 43 RGI60-17.02642
44 44 RGI60-17.02665
45 45 RGI60-17.02666
46 46 RGI60-17.02671
47 47 RGI60-17.02674
48 48 RGI60-

366 366 RGI60-17.10829
367 367 RGI60-17.10880
368 368 RGI60-17.10888
369 369 RGI60-17.10900
370 370 RGI60-17.10919
371 371 RGI60-17.10930
372 372 RGI60-17.10944
373 373 RGI60-17.10965
374 374 RGI60-17.10967
375 375 RGI60-17.10979
376 376 RGI60-17.11011
377 377 RGI60-17.11068
378 378 RGI60-17.11081
379 379 RGI60-17.11138
380 380 RGI60-17.11152
381 381 RGI60-17.11195
382 382 RGI60-17.11313
383 383 RGI60-17.11365
384 384 RGI60-17.11394
385 385 RGI60-17.11414
386 386 RGI60-17.11490
387 387 RGI60-17.11632
388 388 RGI60-17.11646
389 389 RGI60-17.11720
390 390 RGI60-17.11742
391 391 RGI60-17.11765
392 392 RGI60-17.11910
393 393 RGI60-17.12012
394 394 RGI60-17.12031
395 395 RGI60-17.12051
396 396 RGI60-17.12085
397 397 RGI60-17.12094
398 398 RGI60-17.12149
399 399 RGI60-17.12178
400 400 RGI60-17.12195
401 401 RGI60-17.12227
402 402 RGI60-17.12273
403 403 RGI60-17.12294
404 404 RGI60-17.12370
405 405 RGI60-17.12371
406 406 RGI60-17.12378
407 407 RGI60-17.12484
408 408 RGI60-17.12495
409 409 RGI

In [7]:
print('\n\nDONE!\n\n')



DONE!




In [ ]:
# ===== SHEAN ESTIMATE OF FLUX DIVERGENCE QUICKLY ======
#                 if gf.H is not None:
#                     #Compute flux
#                     gf.Q = gf.H * debris_prms.v_col_f * np.array([gf.vx, gf.vy])
#                     #Note: np.gradient returns derivatives relative to axis number, so (y, x) in this case
#                     #Want x-derivative of x component
#                     gf.divQ = np.gradient(gf.Q[0])[1] + np.gradient(gf.Q[1])[0]
# #                     gf.divQ = gf.H*(np.gradient(v_col_f*gf.vx)[1] + np.gradient(v_col_f*gf.vy)[0]) \
# #                             + v_col_f*gf.vx*(np.gradient(gf.H)[1]) + v_col_f*gf.vy*(np.gradient(gf.H)[0])
#                     #Should smooth divQ, better handling of data gaps

In [ ]:
# ===== OLD CHECK DEM FOR ERRORS AND REPLACE SCRIPT (no longer needed with OGGM processing) =====
#         #Create buffer around glacier polygon
#         glac_geom_buff = gf.glac_geom.Buffer(debris_prms.buff_dist)
#         #This is False over glacier polygon surface, True elsewhere - can be applied directly
#         glac_geom_buff_mask = geolib.geom2mask(glac_geom_buff, ds_dict['ice_thick'])
        
#             # ds masks
#             ds_list_masked = [iolib.ds_getma(i) for i in ds_list]
#             dem1 = np.ma.masked_less_equal(ds_list_masked[0], 0)
#             dems_mask = dem1.mask
#             if verbose:
#                 print('list of datasets:', len(ds_list_masked), fn_dict.values())

#             #Combine to identify ~1 km buffer around glacier polygon over static rock
#             static_buffer_mask = np.ma.mask_or(~glac_shp_lyr_mask, glac_geom_buff_mask)
#             static_shp_lyr_mask = np.ma.mask_or(static_buffer_mask, dems_mask)
        
        
#             # Check if DEM has huge errors or not - replace if necessary
#             if input.roi in ['01']:

#                 gf.z1_check = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
#                 if gf.z1_check.min() < 0:

#                     # Add backup DEM for regions with known poor quality (ex. Alaska)
#                     print('switching DEMs')
#                     fn_dict['z1_backup'] = input.z1_backup_dict[input.roi]
#                     # Warp everything to common res/extent/proj (a second time)
#                     ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, \
#                             extent=warp_extent, t_srs=aea_srs, verbose=verbose, \
#                             r='cubic')
#                     ds_dict = dict(zip(fn_dict.keys(), ds_list))

#                     if verbose:
#                         print(ds_list)
#                         print(fn_dict.keys())

#                     # ds masks
#                     ds_list_masked = [iolib.ds_getma(i) for i in ds_list]
#                     dem1 = np.ma.masked_less_equal(ds_list_masked[-1], 0)
#                     dems_mask = dem1.mask
#                     if verbose:
#                         print('list of datasets:', len(ds_list_masked), fn_dict.values())

#                     #Combine to identify ~1 km buffer around glacier polygon over static rock
#                     static_buffer_mask = np.ma.mask_or(~glac_shp_lyr_mask, glac_geom_buff_mask)
#                     static_shp_lyr_mask = np.ma.mask_or(static_buffer_mask, dems_mask)

#                     #This is False over glacier polygon surface, True elsewhere - can be applied directly
#                     glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1_backup'])
#                     gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1_backup']), mask=glac_geom_mask)
#                     #gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)

#                     # Debris cover
#                     dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
#                     gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1_backup']), mask=dc_mask)